# Example of how to use Deep Learning for text classification

Click [Text Classification Example](https://colab.research.google.com/github/magic-lantern/coprh-nlp-2021/blob/main/text_classification_colab.ipynb) to open this notebook in Google Colab - no local setup required, results saved to your Google Drive.


In Google Colab, make sure you change the Runtime to a GPU instance - that will result vastly improved runtimes for deep learning steps. On CPU based hardware some steps take 40 minutes versus 1 minute per training epoch on systems with a GPU. Steps to update runtime:

    Runtime > Change runtime type > Select 'GPU' in Hardware accelerator box

Click 'Save' to switch from Default runtime to GPU accelerated runtime.

In [ ]:
# this cell makes sure Google Colab version has latest software dependencies
!pip install -Uqq fastbook fastai

In [ ]:
import fastbook
import gc

In [ ]:
# this cell will setup a path that allows you to load and save files to your google drive
# you will be prompted to log in with a Google enabled account.
fastbook.setup_book()

In [ ]:
from fastai.text.all import *

In [ ]:
Path.cwd()

In [ ]:
data_path = fastbook.gdrive / Path('data')
data_file = data_path / 'mtsamples.csv'

In [ ]:
data_file

In [ ]:
if data_file.is_file():
    print('Already downloaded')
else:
    print('downloading data file')
    download_data(
        url='https://github.com/socd06/medical-nlp/raw/master/data/mtsamples.csv',
        fname = data_file
    )

In [ ]:
mtsamples = pd.read_csv(str(data_path) + '/mtsamples.csv')
mtsamples.head()

In [ ]:
# find rows where number of words is very low

In [ ]:
word_len = mtsamples.transcription.str.split(' ').str.len()

In [ ]:
word_len.median()

In [ ]:
word_len.min()

In [ ]:
word_len.max()

In [ ]:
mtsamples.medical_specialty.value_counts()

This is one way to load the data for training

```python
TextDataLoaders.from_df(mtsamples,
                        valid_pct=0.2,
                        seed=42,
                        text_col='transcription',
                        label_col='medical_specialty',
                        seq_len=72)
```

In [ ]:
# dls_lm = DataBlock(
#     blocks=(TextBlock.from_df('transcription', is_lm=True), CategoryBlock),
#     get_x=ColReader('transcription'), 
#     get_y=ColReader('medical_specialty'),
#     splitter=TrainTestSplitter(test_size=0.2,
#                                random_state=42,
#                                stratify=mtsamples.medical_specialty)
# )

In [ ]:
dls_lm = DataBlock(
    blocks=TextBlock.from_df('transcription', is_lm=True),
    get_x=ColReader('text'),
    splitter=TrainTestSplitter(test_size=0.2,
                               random_state=42)
)

In [ ]:
# if changes in the above were made (such as batch size), it may be beneficial to purge GPU memory
# gc.collect()
# torch.cuda.empty_cache()
# gc.collect()
# torch.cuda.empty_cache()

In [ ]:
# This cell takes 1 - 2 minutes
# when using seq_len, bs of 128 fits in GPU memory, bs 256 does not fit
dls_lm = dls_lm.dataloaders(mtsamples, bs=128, seq_len=256)

In [ ]:
dls_lm.show_batch(max_n=2)

In [ ]:
learn = language_model_learner(
    dls_lm,
    AWD_LSTM,
    model_dir=fastbook.gdrive / Path('data'),
    drop_mult=0.3,
    metrics=[accuracy, Perplexity()]).to_fp16()

In [ ]:
# on macbook this takes 40 minues
# on GPU machine this takes about 1 minute
learn.fit_one_cycle(1, 2e-2)

In [ ]:
learn.save('1stepoch')
# can load already saved trained model with
# learn.load('1stepoch')

In [ ]:
# if changes in the above were made (such as batch size), it may be beneficial to purge GPU memory
# gc.collect()
# torch.cuda.empty_cache()
# gc.collect()
# torch.cuda.empty_cache()

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(10, 2e-3)

In [ ]:
learn.save_encoder('finetuned')
# can load with
# learn.load('finetuned')

In [ ]:
learn.load('somef')

In [ ]:
if (Path('/content/gdrive/My Drive/data/finetuned.pth').is_file()):
  print('Loading previously saved language model learner')
  learn.load()

In [ ]:
dls_lm = DataBlock(
    blocks=TextBlock.from_df('transcription', is_lm=True),
    get_x=ColReader('text'),
    #get_y='medical_specialty',
    splitter=TrainTestSplitter(test_size=0.2,
                               random_state=42,
                               stratify=mtsamples.medical_specialty)
)